<a href="https://colab.research.google.com/github/josh-kindhart/Python-Practice/blob/main/First_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install keras
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1500)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv('medical_costs.csv')

In [3]:
display(df.head())

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,southwest,16884.92400
1,18,0,33.770,1,0,southeast,1725.55230
2,28,0,33.000,3,0,southeast,4449.46200
3,33,0,22.705,0,0,0rthwest,21984.47061
4,32,0,28.880,0,0,0rthwest,3866.85520


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   int64  
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   int64  
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 73.3+ KB
None


In [5]:
print(df.region.unique())

['southwest' 'southeast' '0rthwest' '0rtheast']


In [6]:
df.replace('0rthwest', 'northwest', inplace=True)
df.replace('0rtheast','northeast', inplace=True)

df.sex.replace(1, 'female', inplace=True)
df.sex.replace(0, 'male', inplace=True)

df.sex = pd.Categorical(df.sex, ['male','female'], ordered=False)
df.region = pd.Categorical(df.region, list(df.region.unique()), ordered=False)

In [7]:
features = df.iloc[:, 0:6]
labels = df.iloc[:, -1]

In [8]:
features = pd.get_dummies(features)

In [9]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

In [10]:
ct = ColumnTransformer([('normalize', Normalizer(), ['age','bmi','children'])], remainder='passthrough')
features_train = ct.fit_transform(features_train)
features_test = ct.transform(features_test)

In [11]:
def design_model(features_train):
  model = Sequential(name="myfirstmodel")
  num_features = features_train.shape[1]
  input = InputLayer(input_shape=(num_features,))
  model.add(input)
  model.add(Dense(128, activation = 'relu'))
  model.add(Dense(1))
  return model

In [12]:
model = design_model(features_train)
opt = Adam(learning_rate=0.01)
model.compile(loss='mse', metrics=['mae'], optimizer=opt)
model.fit(features_train, labels_train, batch_size=1, epochs=40)

Epoch 1/40
896/896 [==============================] - 3s 2ms/step - loss: 213235616.0000 - mae: 9814.1396
Epoch 2/40
896/896 [==============================] - 2s 2ms/step - loss: 121876728.0000 - mae: 8177.8550
Epoch 3/40
896/896 [==============================] - 2s 2ms/step - loss: 108390208.0000 - mae: 7733.2798
Epoch 4/40
896/896 [==============================] - 2s 2ms/step - loss: 96242800.0000 - mae: 7418.9829
Epoch 5/40
896/896 [==============================] - 2s 2ms/step - loss: 84712760.0000 - mae: 7064.1143
Epoch 6/40
896/896 [==============================] - 2s 2ms/step - loss: 74719896.0000 - mae: 6527.3232
Epoch 7/40
896/896 [==============================] - 2s 2ms/step - loss: 66366248.0000 - mae: 6230.8760
Epoch 8/40
896/896 [==============================] - 2s 3ms/step - loss: 60647808.0000 - mae: 6008.3794
Epoch 9/40
896/896 [==============================] - 2s 3ms/step - loss: 57140800.0000 - mae: 5745.4971
Epoch 10/40
896/896 [==============================]

In [13]:
val_mse, val_mae = model.evaluate(features_test, labels_test, verbose=0)

In [14]:
print(f'MSE: {val_mse}, MAE: {val_mae}')

MSE: 47847372.0, MAE: 4821.68896484375


In [15]:
print(model.summary())

Model: "myfirstmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________
None
